### IMPORT NECESSARY PACKAGES 

In [59]:
import pandas as pd
import sqlite3

### LOAD THE DATA SET ONLY 2010-2011 SHEET

In [17]:
df = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom


### CLEAN AND PREPARE THE DATA 

In [16]:
#Clean the columns 
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
print(df.columns.tolist())
 
#Drop rows with missing Customer_ID
df = df.dropna(subset=["customer_id"])

#Convert date columns 
df["invoicedate"] = pd.to_datetime(df["invoicedate"])

#Ensure the correct data type
df["customer_id"] = df["customer_id"].astype(int)
df["quantity"] = df["quantity"].astype(int)
df["price"] = df["price"].astype(float)

#Quick check
df.info()

['invoice', 'stockcode', 'description', 'quantity', 'invoicedate', 'price', 'customer_id', 'country']
<class 'pandas.core.frame.DataFrame'>
Index: 406830 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      406830 non-null  object        
 1   stockcode    406830 non-null  object        
 2   description  406830 non-null  object        
 3   quantity     406830 non-null  int64         
 4   invoicedate  406830 non-null  datetime64[ns]
 5   price        406830 non-null  float64       
 6   customer_id  406830 non-null  int64         
 7   country      406830 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 27.9+ MB


### EXPORT SQLITE

In [18]:
#Connect to SQLite (create a retail.db)
conn = sqlite3.connect("retail.db")

#Write to SQL Table
df.to_sql("transactions", conn, if_exists="replace", index=False)

#Confirm if the table is created
print(pd.read_sql("SELECT COUNT(*) FROM transactions", conn))

   COUNT(*)
0    541910


### RUN SQL QUERY

In [19]:
#Total sales by country
query = """
SELECT country, SUM(quantity * price) AS total_sales
FROM transactions
GROUP BY country
ORDER BY total_sales DESC
LIMIT 10;
"""

pd.read_sql(query, conn)

,Country,total_sales
0,United Kingdom,8.187806e+06
1,Netherlands,2.846615e+05
2,EIRE,2.632768e+05
3,Germany,2.216982e+05
4,France,1.974219e+05
5,Australia,1.370773e+05
6,Switzerland,5.638535e+04
7,Spain,5.477458e+04
8,Belgium,4.091096e+04
9,Sweden,3.659591e+04


### RFM ANALYSIS USING SQL (SQLite)
### (R)ecency: How recently a customer made their last purchase
### (F)requency: How often a customer purchase 
### (M)onetary: How much does a customer spend in total 

### Find the most invoice date:

In [34]:
query = """
SELECT MAX(invoicedate) AS last_order_date
FROM transactions:
"""

# Full RFM Query

In [50]:
rfm_query = """
SELECT
    "Customer ID" AS customer_id,
    julianday('2011-12-10') - julianday(MAX(InvoiceDate)) AS recency,
    COUNT(DISTINCT Invoice) AS frequency,
    ROUND(SUM(Quantity * Price), 2) AS monetary
FROM transactions
GROUP BY "Customer ID"
HAVING monetary > 0;
"""

rfm_df = pd.read_sql(rfm_query, conn)
rfm_df.head()


,customer_id,recency,frequency,monetary
0,NaN,0.565278,3710,1447682.12
1,12347.0,2.338889,7,4310.00
2,12348.0,75.449306,4,1797.24
3,12349.0,18.589583,1,1757.55
4,12350.0,310.332639,1,334.40


### RFM SCORING AND SEGMENTATION IN PYTHON

In [54]:
#Drop NaN for customer_id 0
rfm_df = rfm_df.dropna(subset=["customer_id"])

#Convert customer_id into (int)
rfm_df["customer_id"] = rfm_df["customer_id"].astype(int)

#Create RFM score (1=worst, 4=best)
rfm_df["R_score"] = pd.qcut(rfm_df["recency"], 4, labels=[4, 3, 2, 1]) # lower recency=better score
rfm_df["F_score"] = pd.qcut(rfm_df["frequency"].rank(method="first"), 4, labels=[1, 2, 3, 4]) # higher frequency=better score
rfm_df["M_score"] = pd.qcut(rfm_df["monetary"], 4, labels=[1, 2, 3, 4]) # higher monetary=better score

#combine into RFM score 
rfm_df["RFM_Score"] = rfm_df["R_score"].astype(str) + rfm_df["F_score"].astype(str) + rfm_df["M_score"].astype(str)
rfm_df.head(10)


,customer_id,recency,frequency,monetary,R_score,F_score,M_score,RFM_Score
1,12347,2.338889,7,4310.00,4,4,4,444
2,12348,75.449306,4,1797.24,2,3,4,234
3,12349,18.589583,1,1757.55,3,1,4,314
4,12350,310.332639,1,334.40,1,1,2,112
5,12352,36.390972,11,1545.41,3,4,3,343
6,12353,204.259028,1,89.00,1,1,1,111
7,12354,232.450694,1,1079.40,1,1,3,113
8,12355,214.424306,1,459.40,1,1,2,112
9,12356,22.638889,3,2811.43,3,2,4,324
10,12357,33.328472,1,6207.67,3,1,4,314


### LABEL CUSTOMERS BY SEGMENTS 

#Segment Mapping Logic

| Segment                | Logic               |
| ---------------------- | ------------------- |
| **Champions**          | R ≥ 4, F ≥ 4, M ≥ 4 |
| **Loyal Customers**    | F ≥ 4               |
| **Potential Loyalist** | R ≥ 3, F ≥ 3        |
| **Recent Customers**   | R = 5, F ≤ 2        |
| **Promising**          | R = 4, F ≤ 2        |
| **Needs Attention**    | R = 2 or 3, F ≤ 3   |
| **At Risk**            | R ≤ 2, F ≥ 3        |
| **Can't Lose Them**    | R ≤ 2, F ≥ 4, M ≥ 4 |
| **Lost**               | R = 1, F ≤ 2, M ≤ 2 |


In [58]:
#Implementing the mapping in Python
def assign_segment(row):
    r = row["R_score"]
    f = row["F_score"]
    m = row["M_score"]

    if r >= 4 and f >= 4 and m >= 4:
        return "Champion"
    elif f >= 4 and r >= 2:
        return "Loyal Customer"
    elif r >= 3 and f >= 3:
        return "Potential loyalist"
    elif r == 5 and f <= 2:
        return "Recent Customers"
    elif r == 4 and f <= 2:
        return "Promising"
    elif (r == 2 or r == 3) and f <= 3:
        return "Need More Attention"
    elif r <= 2 and f >= 3:
        return "At Risk"
    elif r <= 2 and f >= 4 and m >= 4:
        return "Cant Lose Them" 
    elif r == 1 and f <= 2 and m <= 2:
        return "Lost"
    else:
        return "Others"

#Apply segmentation
rfm_df["Segment"] = rfm_df.apply(assign_segment, axis=1)

rfm_df[["customer_id", "R_score", "F_score", "M_score", "RFM_Score", "Segment"]].head(10)
    
    

,customer_id,R_score,F_score,M_score,RFM_Score,Segment
1,12347,4,4,4,444,Champion
2,12348,2,3,4,234,Need More Attention
3,12349,3,1,4,314,Need More Attention
4,12350,1,1,2,112,Lost
5,12352,3,4,3,343,Loyal Customer
6,12353,1,1,1,111,Lost
7,12354,1,1,3,113,Others
8,12355,1,1,2,112,Lost
9,12356,3,2,4,324,Need More Attention
10,12357,3,1,4,314,Need More Attention


In [61]:
rfm_df.to_csv("rfm_segment.csv", index=False)